In [ ]:
from __future__ import print_function

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import torchvision
import numpy as np
import pandas as pd
import tensorflow as tf
from os.path import join, exists, expanduser
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation, \
    Input, merge, Lambda
print(torch.__version__)

In [ ]:
use_cuda = torch.cuda.is_available()

In [ ]:
from dataset import dataset
from AlexNet import AlexNet
from train_test import start_train_test

In [ ]:
trainloader, validateloader, testloader, outputs, inputs = dataset('dog-breed')
print ('Output classes: {}\nInput channels: {}'.format(outputs, inputs))

In [ ]:
print(len(trainloader))

In [ ]:
print(len(validateloader))

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputdata, classes = next(iter(trainloader))
# Make a grid from batch
out = torchvision.utils.make_grid(inputdata)

imshow(out)


In [ ]:
net = AlexNet(num_classes = outputs, inputs=inputs)
file_name = 'alexnet-'

In [ ]:
if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
train_loss,train_prodicted, test_loss,test_prodicted = start_train_test(net, trainloader, trainloader, criterion)

In [ ]:
plt.plot(train_loss)
plt.ylabel('Train Loss')
plt.show()
print(train_prodicted)

In [ ]:
plt.plot(test_loss)
plt.ylabel('Test Loss')
plt.show()